<a href="https://colab.research.google.com/github/ftirelo/ftirelo-expts/blob/main/Generate_emotions_form.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Pip installs

!pip install pyyaml==6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0.2
    Uninstalling PyYAML-6.0.2:
      Successfully uninstalled PyYAML-6.0.2


In [2]:
#@title Imports

from __future__ import print_function

import re
import yaml

from google.colab import auth
from googleapiclient.discovery import build


In [3]:
#@title Authenticate and create services

PROJECT_ID = 'project-tracking-336cd'  #@param

# os.environ['GOOGLE_CLOUD_PROJECT'] = 'project-tracking-336cd'
auth.authenticate_user(project_id=PROJECT_ID)

drive_service = build('drive', 'v3')
forms_service = build('forms', 'v1')


In [37]:
#@title Google Forms functions

def create_form(title, description):
  new_form = {
      'info': {
          'title': title,
          'document_title': title
      }
  }
  result = forms_service.forms().create(body=new_form).execute()
  form_id = result['formId']

  result = forms_service.forms().batchUpdate(
      formId=form_id,
      body={'requests': [
          {
              'updateFormInfo': {
                  'info': {
                      'description': description
                  },
                  'updateMask': 'description'
              }
          }
      ]}
  ).execute()

  return form_id

class ChoiceQuestion:
  def __init__(self, form_type, title):
    self.form_type = form_type
    self.title = title
    self.required = False
    self.options = []
    self.has_other = False
    self.shuffle = False

  def add_options(self, values):
    for v in values:
      if v == 'OTHER':
        self.add_other_option()
      else:
        self.options.append({'value': v})
    return self

  def add_option(self, value):
    self.options.append({'value': value})
    return self

  def add_other_option(self):
    self.has_other = True
    self.options.append({'is_other': True})
    return self

  def set_required(self):
    self.required = True
    return self

  def set_shuffle(self):
    self.shuffle = True
    return self

  def build_request(self):
    if len(self.options) == 0:
      raise Exception('No options added')

    return {
        "title": self.title,
        "questionItem": {
            "question": {
                "required": self.required,
                "choiceQuestion": {
                    "type": self.form_type,
                    "shuffle": self.shuffle,
                    "options": self.options
                }
            }
        }
    }

class RadiobuttonQuestion(ChoiceQuestion):
  def __init__(self, title):
    ChoiceQuestion.__init__(self, 'RADIO', title)

class CheckboxQuestion(ChoiceQuestion):
  def __init__(self, title):
    ChoiceQuestion.__init__(self, 'CHECKBOX', title)

class Section:
  def __init__(self, title):
    self.title = title

  def build_request(self):
    return {
        "title": self.title,
        "pageBreakItem": {}
    }

class Form:
  def __init__(self, title, description):
    self.form_id = None
    self.title = title
    self.description = description
    self.items = []

  def get_form_url(self):
    if not self.form_id:
      raise Exception('Form not saved')

    return f'https://docs.google.com/forms/d/{self.form_id}/edit'

  def add_item(self, item):
    if self.form_id:
      raise Exception('Form already saved')

    self.items.append(item)

  def save(self):
    if self.form_id:
      raise Exception('Form already saved')

    self.form_id = create_form(self.title, self.description)
    for location, item in enumerate(self.items):
      try:
        request = {
                    "createItem": {
                        "item": item,
                        "location": {
                            "index": location
                        }
                    }
                }
        result = forms_service.forms().batchUpdate(
            formId=self.form_id,
            body={'requests': [request]}
        ).execute()
      except Exception as e:
        print(e)
        print(request)
        break


In [39]:
#@title Load file from YAML

def clean_and_load_yaml(filename):
    with open(filename, 'r') as file:
        content = file.read()

    # Remove invalid YAML characters using a regular expression.
    cleaned_content = re.sub(r"[^\w\s.,;:'\"!?]", "", content)
    content_without_comments = re.sub(r"#.*", "", content)

    try:
        # Load the cleaned YAML content into a Python object.
        data = yaml.safe_load(content_without_comments)
        return data
    except yaml.YAMLError as exc:
        print(f"Error parsing YAML: {exc}")
        return None

# Example usage:
filename = '/content/emotions.yaml'
sections = clean_and_load_yaml(filename)

In [50]:
#@title Create emotions form

TITLE = 'Emotion words analysis'  #@param
DESCRIPTION = 'Collecting words and feelings that contribute to emotions'  #@param
INITIAL_QUESTION_TITLE = 'Choose one of the emotions above or a random number: https://www.google.com/search?q=random+number+between+1+and+10&oq=random+number+between+1+and+10'

form = Form(TITLE, DESCRIPTION)

form.add_item(
    RadiobuttonQuestion(INITIAL_QUESTION_TITLE)
        .set_required()
        .add_options([str(p + 1) + ': ' + s['name'] for p, s in enumerate(sections)])
        .build_request())

for section in sections:
  emotion = section['name']
  form.add_item(Section(emotion).build_request())
  words = section['words']
  form.add_item(
      CheckboxQuestion(f'Words for {emotion}')
          .set_required()
          .set_shuffle()
          .add_options(words)
          .build_request())

  for question in section['questions']:
    title = question['title']
    options = question['options']
    form.add_item(
        CheckboxQuestion(title)
            .set_required()
            .add_options(options)
            .build_request())

form.save()
print(form.get_form_url())

https://docs.google.com/forms/d/1A-oGalJoiSerJrWL6XYfcDQ7RTyqEziBAqelrzq_Lzw/edit
